In [1]:
# https://www.kaggle.com/kernels/scriptcontent/11511967/notebook

import string
import numpy as np
import pandas as pd
# from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast
import re
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
# from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import nltk
from nltk.tag import tnt
from nltk.corpus import stopwords
from nltk.corpus import indian
from nltk.tokenize import word_tokenize,sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

import stanfordnlp

# from bokeh.plotting import figure, output_file, show
# from bokeh.models import Label
# from bokeh.io import output_notebook
# output_notebook()

%matplotlib inline

C:\Users\Sarthak\Anaconda3\envs\gen_env\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
#nltk.download()

In [3]:
# stanfordNLP initialization

config = {
    "processors" : "tokenize,mwt,lemma,pos",
    "lang" : "hi",
    # Processor-specific arguments are set with keys "{processor_name}_{argument_name}"
    'tokenize_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tokenizer.pt', 
# 	'mwt_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\',
	'pos_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tagger.pt',
	'pos_pretrain_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb.pretrain.pt',
	'lemma_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_lemmatizer.pt'
# 	'depparse_model_path': './fr_gsd_models/fr_gsd_parser.pt',
# 	'depparse_pretrain_path': './fr_gsd_models/fr_gsd.pretrain.pt'
}

nlp = stanfordnlp.Pipeline(**config) # Initialize the pipeline using a configuration dict
# nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")

<>:7: DeprecationWarning: invalid escape sequence \D
<>:9: DeprecationWarning: invalid escape sequence \D
<>:10: DeprecationWarning: invalid escape sequence \D
<>:11: DeprecationWarning: invalid escape sequence \D
<>:7: DeprecationWarning: invalid escape sequence \D
<>:9: DeprecationWarning: invalid escape sequence \D
<>:10: DeprecationWarning: invalid escape sequence \D
<>:11: DeprecationWarning: invalid escape sequence \D
<>:7: DeprecationWarning: invalid escape sequence \D
<>:9: DeprecationWarning: invalid escape sequence \D
<>:10: DeprecationWarning: invalid escape sequence \D
<>:11: DeprecationWarning: invalid escape sequence \D
<ipython-input-3-c25034751c4f>:7: DeprecationWarning: invalid escape sequence \D
  'tokenize_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tokenizer.pt',
<ipython-input-3-c25034751c4f>:9: DeprecationWarning: invalid escape sequence \D
  'pos_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tagger.pt',
<ipython-input-3

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_lemmatizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: pos
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tagger.pt', 'pretrain_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Done loading processors!
---


In [4]:
datafile = 'D:/Workspace/Projects/ramayanaocr/src/ramayanaocr/data/hindi.txt'
with open(datafile,'r',encoding='utf-8') as f:
    text = f.read()
    text = text.split("॥")

In [5]:
custom = [word for word in tqdm(text) if not re.findall("^\s?\d+\s?",word)]

<>:1: DeprecationWarning: invalid escape sequence \s
<>:1: DeprecationWarning: invalid escape sequence \s
<>:1: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-110b5ca9b6e7>:1: DeprecationWarning: invalid escape sequence \s
  custom = [word for word in tqdm(text) if not re.findall("^\s?\d+\s?",word)]
100%|██████████████████████████████████████████████████████████████████████████| 1290/1290 [00:00<00:00, 184733.25it/s]


In [6]:
text[0:20]

[' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया ',
 '१७',
 ' घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया ',
 ' १ ',
 'धानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया ',
 ' २ ',
 " उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ ",
 ' एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे ',
 '५',
 ' वहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा ',
 '६',
 ' राक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा ',
 ' ७',
 ' तव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है',
 '८',
 ' विशाल नेत्रोंवाली वैदेही सी

In [7]:
word = '१७ ९) सीता को रावण के भन्ता'
re.findall("\d", word)

<>:2: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \d
<ipython-input-7-68254fe98b29>:2: DeprecationWarning: invalid escape sequence \d
  re.findall("\d", word)


['१', '७', '९']

In [8]:
new_text = []
i = 0
for word in tqdm(text):
#     print(i,word)
    if re.findall("^\s?\d+\s?",word):
        continue
    else:   
        new_text.append(word)
        
#or 

"""
new_text = [word for word in tqdm(text) if not re.findall("^\s?\d+\s?",word)]
"""

<>:5: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<ipython-input-8-16961d38465f>:5: DeprecationWarning: invalid escape sequence \s
  if re.findall("^\s?\d+\s?",word):
<ipython-input-8-16961d38465f>:14: DeprecationWarning: invalid escape sequence \s
  """
100%|██████████████████████████████████████████████████████████████████████████| 1290/1290 [00:00<00:00, 184752.17it/s]


'\nnew_text = [word for word in tqdm(text) if not re.findall("^\\s?\\d+\\s?",word)]\n'

In [9]:
len(new_text)

667

In [10]:
text_df = pd.DataFrame(new_text)

In [11]:
text_df.head()

,0
0,वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास...
1,घ४-९) सीता को रावण के भन्तापुर में ढूंढना : व...
2,धानरराज के उस हितैषी ने लंका नगरी में प्रवेश क...
3,उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के ...
4,एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्...


In [12]:
# hindi stopwords

stop_words_df = pd.read_csv("D:/Data/CaseStudies-2/data/stopwords.txt", header = None)
stop_words = list(set(stop_words_df.values.reshape(1,-1).tolist()[0]))
stop_words.extend(["।", "।।", ")", "(", ",","","हे" ])

In [13]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data)
#     print(vectorized_headlines)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0] for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [14]:
count_vectorizer = CountVectorizer(stop_words=stop_words)

In [15]:
count_vectorizer.fit_transform(new_text)

C:\Users\Sarthak\Anaconda3\envs\gen_env\lib\site-packages\scikit_learn-0.22.1-py3.6-win-amd64.egg\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['अथव', 'अध', 'अन', 'अपन', 'अभ', 'आग', 'आद', 'आपक', 'इत', 'इनक', 'इसक', 'इसम', 'इसल', 'उनक', 'उसक', 'एव', 'ऐस', 'कभ', 'करत', 'करन', 'कह', 'कहत', 'गय', 'जबक', 'जर', 'जह', 'झक', 'तथ', 'तन', 'तर', 'दर', 'धर', 'नस', 'नह', 'पड', 'पहल', 'बड', 'बन', 'बह', 'यत', 'यद', 'यम', 'रख', 'रत', 'रह', 'रहत', 'लक', 'वय', 'वर', 'वग़', 'सक', 'सकत', 'सबस', 'सभ', 'सम', 'सर', 'सस', 'हमन', 'हर'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<667x1070 sparse matrix of type '<class 'numpy.int64'>'
	with 5458 stored elements in Compressed Sparse Row format>

In [16]:
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=new_text)

# Playground

In [17]:
# nltk pos tag

train_data = indian.tagged_sents('hindi.pos')
tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)

In [18]:
def create_str_from_list(original_read_text):
    prepared_text = ""
    for line in original_read_text:
        line = line.split()
#       print(line)
        tmp_line = " ".join(line)
        prepared_text += " "+tmp_line
    return prepared_text

x = create_str_from_list(new_text)

In [19]:
x[0:1000]

' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया धानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया उस समय सुन्दर \'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे वहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा राक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा तव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है विशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उसके चारों ओर घूमा ।। ९ तब वह उस सुन्दर पड़ी मा

In [20]:
# mapped = list(map(lambda each:each.strip("\n"), sentence_list))
tagged_words = (tnt_pos_tagger.tag(nltk.word_tokenize(x[0:1000])))
print(tagged_words)

[('वानर', 'Unk'), ('हनुमान', 'Unk'), ('सीमा', 'NN'), ('के', 'PREP'), ('देखने', 'Unk'), ('की', 'PREP'), ('उत्कण्ठा', 'Unk'), ('येसूयास्त', 'Unk'), ('की', 'PREP'), ('प्रतीक्षा', 'Unk'), ('करता', 'VFM'), ('भया', 'Unk'), ('घ४-९', 'Unk'), (')', 'SYM'), ('सीता', 'Unk'), ('को', 'PREP'), ('रावण', 'Unk'), ('के', 'PREP'), ('भन्तापुर', 'Unk'), ('में', 'PREP'), ('ढूंढना', 'Unk'), (':', 'SYM'), ('वह', 'PRP'), ('महावीर्य', 'Unk'), ('महान', 'JJ'), ('हृदय', 'Unk'), ('पाछा', 'Unk'), ('वानर', 'Unk'), ('श्रेष्ठ', 'Unk'), ('रात', 'Unk'), ('के', 'PREP'), ('समय', 'NN'), ('अद्वार', 'Unk'), ('से', 'PREP'), ('कोट', 'Unk'), ('को', 'PREP'), ('फांद', 'Unk'), ('कर', 'VRB'), ('का', 'PREP'), ('में', 'PREP'), ('प्रविष्ट', 'Unk'), ('हृया', 'Unk'), ('धानरराज', 'Unk'), ('के', 'PREP'), ('उस', 'PRP'), ('हितैषी', 'Unk'), ('ने', 'PREP'), ('लंका', 'Unk'), ('नगरी', 'Unk'), ('में', 'PREP'), ('प्रवेश', 'Unk'), ('करके', 'Unk'), ('मानों', 'Unk'), ('अपना', 'PRP'), ('बायां', 'Unk'), ('पाओं', 'Unk'), ('शत्रु', 'Unk'), ('के', 'PREP')

In [21]:
# gensim sentence to words
# not for hindi

def sent_to_words(sentences):
    for sentence in sentences:
        
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuations

data_words = list(sent_to_words(new_text))

print(data_words[:1])

[['नर', 'हन', 'खन', 'उत', 'कण', 'रत', 'करत', 'भय']]


In [43]:
# word tokenize

words_list = []
for line in new_text:
    words_list = words_list + word_tokenize(line)

In [44]:
words_tokenized = [word_tokenize(line) for line in new_text]

In [45]:
print(words_list == words_tokenized)

False


In [23]:
# Creating Bigram and Trigram Models
# Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.
data_words = words_list
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
# print(trigram_mod[bigram_mod[data_words]])
# print(bigram_mod[data_words])

In [24]:
# Remove Stopwords, Make Bigrams and Lemmatize

# Define functions for stopwords, bigrams, trigrams and lemmatization
# def remove_stopwords(text):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def remove_stopwords(word_tokenized):
    return [word for word in words_list if word not in stop_words]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# Lemmatization

def extract_lemma(doc):
    parsed_text = {'word':[], 'lemma':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
#             print(wrd.text)
            #extract text and lemma
            parsed_text['word'].append(wrd.text)
            parsed_text['lemma'].append(wrd.lemma)
    #return a dataframe
    return pd.DataFrame(parsed_text)

def lemmatization(words_tokenized):
    return [word.lemma for word in words_list] #if token.pos_ in allowed_postags]

# POS tagger

# dictionary that contains pos tags and their explanations

pos_dict = {
'CC': 'coordinating conjunction','CD': 'cardinal digit','DT': 'determiner',
'EX': 'existential there (like: \"there is\" ... think of it like \"there exists\")',
'FW': 'foreign word','IN':  'preposition/subordinating conjunction','JJ': 'adjective \'big\'',
'JJR': 'adjective, comparative \'bigger\'','JJS': 'adjective, superlative \'biggest\'',
'LS': 'list marker 1)','MD': 'modal could, will','NN': 'noun, singular \'desk\'',
'NNS': 'noun plural \'desks\'','NNP': 'proper noun, singular \'Harrison\'',
'NNPS': 'proper noun, plural \'Americans\'','PDT': 'predeterminer \'all the kids\'',
'POS': 'possessive ending parent\'s','PRP': 'personal pronoun I, he, she',
'PRP$': 'possessive pronoun my, his, hers','RB': 'adverb very, silently,',
'RBR': 'adverb, comparative better','RBS': 'adverb, superlative best',
'RP': 'particle give up','TO': 'to go \'to\' the store.','UH': 'interjection errrrrrrrm',
'VB': 'verb, base form take','VBD': 'verb, past tense took',
'VBG': 'verb, gerund/present participle taking','VBN': 'verb, past participle taken',
'VBP': 'verb, sing. present, non-3d take','VBZ': 'verb, 3rd person sing. present takes',
'WDT': 'wh-determiner which','WP': 'wh-pronoun who, what','WP$': 'possessive wh-pronoun whose',
'WRB': 'wh-abverb where, when','QF' : 'quantifier, bahut, thoda, kam (Hindi)','VM' : 'main verb',
'PSP' : 'postposition, common in indian langs','DEM' : 'demonstrative, common in indian langs'
}

# extract parts of speech

def extract_pos(doc):
    parsed_text = {'word':[], 'pos':[], 'exp':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
            if wrd.pos in pos_dict.keys():
                pos_exp = pos_dict[wrd.pos]
            else:
                pos_exp = 'NA'
            parsed_text['word'].append(wrd.text)
            parsed_text['pos'].append(wrd.pos)
            parsed_text['exp'].append(pos_exp)
    #return a dataframe of pos and text
    return pd.DataFrame(parsed_text)

In [25]:
# remove stop_words

words_filtered = words_list[:]
for word in words_list:
    if word in stop_words:        
        words_filtered.remove(word)    
# or

clean_text = remove_stopwords(words_list)        
len(words_list),len(clean_text)

(14481, 8332)

In [26]:
# lemmatization

# def extract_lemma(doc):
#     parsed_text = {'word':[], 'lemma':[]}
#     for sent in doc.sentences:
#         for wrd in sent.words:
# #             print(wrd.text)
#             #extract text and lemma
#             parsed_text['word'].append(wrd.text)
#             parsed_text['lemma'].append(wrd.lemma)
#     #return a dataframe
#     return pd.DataFrame(parsed_text)

def lemmatization(text):
    lemmatized_text = []
    for line in tqdm(text):
        if line not in [""," "] :
            doc = nlp(line)
            for sent in doc.sentences:
                for wrd in sent.words:
                    #extract text and lemma
                    lemmatized_text.append(wrd.lemma)
    return lemmatized_text

In [27]:
lemmatized = lemmatization(new_text)

100%|████████████████████████████████████████████████████████████████████████████████| 667/667 [01:12<00:00,  9.22it/s]


In [28]:
clean_text = remove_stopwords(lemmatized)
len(words_list),len(lemmatized),len(clean_text)

(14481, 14489, 8332)

In [29]:
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in clean_text]
print(stripped[:100])
print(len(clean_text),len(stripped))

# remove all tokens that are not alphabetic
# Doesn't work with Hindi
# words = [word for word in clean_text if word.isalpha()]
# print(words[:100])

['वानर', 'हनुमान', 'सीमा', 'देखने', 'उत्कण्ठा', 'येसूयास्त', 'प्रतीक्षा', 'भया', 'घ४९', 'सीता', 'रावण', 'भन्तापुर', 'ढूंढना', '', 'महावीर्य', 'महान', 'हृदय', 'पाछा', 'वानर', 'श्रेष्ठ', 'रात', 'अद्वार', 'कोट', 'फांद', 'प्रविष्ट', 'हृया', 'धानरराज', 'हितैषी', 'लंका', 'नगरी', 'प्रवेश', 'करके', 'मानों', 'बायां', 'पाओं', 'शत्रु', 'सिर', 'रख', 'सुन्दर', 'सब', 'ओर', 'सजे', 'श्वेत', 'मेघ', 'तुल्य', 'राक्षसों', 'पद्माकार', 'स्वस्तिकाकार', 'वर्धमान', 'घर', 'इंका', 'जगमन', '', 'राधा', 'अर्थ', 'श्रीमान्', 'घूमता', 'हुमा', 'देखता', 'भया', 'आनन्दित', 'भया', '३', '४', 'भवन', 'भवन', 'वानरश्रेष्ठ', 'र', 'विविध', 'भाकृति', 'रूपों', 'भवन', 'देखे', 'राक्षसों', 'घरों', 'उसने', 'अप', '', 'हुओं', 'मन्त्र', 'मुने', 'स्वाध्याय', 'रत', 'राक्षसों', 'देखा', 'राक्षसों', 'घर', 'घर', 'बगीचों', 'देखता', 'बेधड़क', 'महलों', 'पास', 'घूमा', 'तव', 'पवनपुत्र', 'हनुमान', 'भवन', 'श्रेष्ठ', 'देखा']
8332 8332


In [30]:
clean_text == stripped

# Python code to get difference of two lists 
# Not using set() 
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

resultant = Diff(clean_text,stripped)
len(set(resultant))

120

In [31]:
def custom_remove_garbage(original_words_list,list_of_garbage_words):
    tmp_list = [word for word in original_words_list if word not in list_of_garbage_words] # garbage list
    tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers
    tmp_list = [word for word in tmp_list if len(re.findall("[a-zA-Z]+",word))==0] # english alphabets
    return tmp_list

<>:3: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
<ipython-input-31-f12f2bed8c24>:3: DeprecationWarning: invalid escape sequence \d
  tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers


In [32]:
final_text = custom_remove_garbage(stripped,resultant)
len(final_text)

7996

In [33]:
# Create the Dictionary and Corpus needed for Topic Modeling
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.

# Create Dictionary
id2word = corpora.Dictionary([final_text])

# Create Corpus
texts = final_text

# Term Document Frequency
corpus = [id2word.doc2bow(texts)] #for text in texts]

# Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of 
# (word_id, word_frequency).
# For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

# View
# print(corpus[:1])
print(id2word[3]) # word a given id corresponds to

अकामा


In [34]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('अंगकी', 1),
  ('अंगुलिये', 1),
  ('अकम्पन', 1),
  ('अकामा', 1),
  ('अकार्य', 1),
  ('अकेकी', 1),
  ('अकेला', 2),
  ('अकेले', 2),
  ('अक्ष', 2),
  ('अक्षोभ्य', 1),
  ('अखाड़ने', 1),
  ('अग', 1),
  ('अगद', 1),
  ('अगली', 1),
  ('अगस्त्य', 1),
  ('अगोदाली', 1),
  ('अग्नि', 4),
  ('अग्रीय', 1),
  ('अघ', 1),
  ('अघवा', 1),
  ('अङ्गद', 2),
  ('अङ्गनेवाले', 1),
  ('अङ्गों', 2),
  ('अङ्गोवाली', 1),
  ('अचिन्त्य', 2),
  ('अच्छा', 3),
  ('अच्छी', 2),
  ('अजर', 1),
  ('अजितेन्द्रिय', 1),
  ('अटारिये', 1),
  ('अड़', 1),
  ('अति', 3),
  ('अती', 1),
  ('अतीव', 3),
  ('अतुछ', 1),
  ('अतुल', 2),
  ('अत्यन्त', 3),
  ('अत्युक्ति', 1),
  ('अथ', 1),
  ('अथका', 1),
  ('अथाद', 1),
  ('अदभुत', 1),
  ('अदुष्ट', 1),
  ('अदृश्य', 1),
  ('अद्भुत', 3),
  ('अद्रद', 1),
  ('अद्रों', 1),
  ('अद्वार', 1),
  ('अधर्म', 1),
  ('अधार', 1),
  ('अधिक', 1),
  ('अधिपति', 2),
  ('अधीन', 1),
  ('अन', 1),
  ('अनधीख', 1),
  ('अनयों', 1),
  ('अनर्थ', 1),
  ('अनर्थक', 1),
  ('अनवरात्मा', 1),
  ('अनाथ', 1),
  ('अनार्या', 1),
  

In [35]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
# Print the Keyword in the 10 topics
# It means the top 10 keywords that contribute to this topic are: ‘hockey’, ‘pts_pt’, ‘pit’.. and 
# so on and the weight of ‘hockey’ on topic 0 is 0.044.

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"राम" + 0.000*"सीता" + 0.000*"रावण" + 0.000*"करके" + 0.000*"वानर" + '
  '0.000*"सेना" + 0.000*"सब" + 0.000*"वाक्य" + 0.000*"वचन" + 0.000*"हनुमान"'),
 (1,
  '0.000*"राम" + 0.000*"रावण" + 0.000*"सीता" + 0.000*"वचन" + 0.000*"हनुमान" + '
  '0.000*"वानर" + 0.000*"करके" + 0.000*"समुद्र" + 0.000*"सेना" + '
  '0.000*"वाक्य"'),
 (2,
  '0.000*"राम" + 0.000*"रावण" + 0.000*"सीता" + 0.000*"करके" + 0.000*"वचन" + '
  '0.000*"सब" + 0.000*"वानर" + 0.000*"युक्त" + 0.000*"राक्षसों" + '
  '0.000*"वाक्य"'),
 (3,
  '0.000*"सीता" + 0.000*"राम" + 0.000*"रावण" + 0.000*"सब" + 0.000*"हनुमान" + '
  '0.000*"वचन" + 0.000*"करके" + 0.000*"सेना" + 0.000*"समुद्र" + 0.000*"सुनकर"'),
 (4,
  '0.000*"राम" + 0.000*"सीता" + 0.000*"रावण" + 0.000*"सब" + 0.000*"वानर" + '
  '0.000*"वचन" + 0.000*"करके" + 0.000*"विभीषण" + 0.000*"हनुमान" + '
  '0.000*"समुद्र"'),
 (5,
  '0.000*"राम" + 0.000*"सीता" + 0.000*"रावण" + 0.000*"सब" + 0.000*"वानर" + '
  '0.000*"करके" + 0.000*"सेना" + 0.000*"समुद्र" + 0.000*"सुनकर" + '
  '0.000

In [37]:
# Compute Model Perplexity and Coherence Score

# Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is.
# In my experience, topic coherence score, in particular, has been more helpful.

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=final_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.813763725692


C:\Users\Sarthak\Anaconda3\envs\gen_env\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
C:\Users\Sarthak\Anaconda3\envs\gen_env\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))



Coherence Score:  nan


In [38]:
# Visualize the topics-keywords

# pyLDAvis package’s interactive chart is designed to work well with jupyter notebooks.
# Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Sarthak\Anaconda3\envs\gen_env\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.109723  0.000045       1        1  98.908936
15     0.093436  0.000056       2        1   1.087826
8      0.000976 -0.000478       3        1   0.000180
2      0.000974 -0.000457       4        1   0.000180
12     0.000949 -0.000293       5        1   0.000180
14     0.000936 -0.000215       6        1   0.000180
3      0.000939 -0.000236       7        1   0.000180
9      0.000930 -0.000173       8        1   0.000180
6      0.000927 -0.000155       9        1   0.000180
4      0.000918 -0.000086      10        1   0.000180
10     0.000903  0.000007      11        1   0.000180
11     0.000897  0.000043      12        1   0.000180
0      0.000899  0.000035      13        1   0.000180
19     0.000897  0.000039      14        1   0.000180
16     0.000875  0.000187      15        1   0.000180
17     0.000869  0.000229      16        1   0.000180
13     0.000862  0.000275      17        1   0.000180
5      0.000858  0.000305      18        1   0.000180
1      0.000840  0.000425      19        1   0.000180
7      0.000836  0.000445      20        1   0.000180, topic_info=     Category        Freq    Term       Total  loglift  logprob
2874  Default  114.000000     राम  114.000000  30.0000  30.0000
3603  Default   90.000000    सीता   90.000000  29.0000  29.0000
2897  Default   87.000000    रावण   87.000000  28.0000  28.0000
3032  Default   48.000000     वचन   48.000000  27.0000  27.0000
3086  Default   45.000000    वानर   45.000000  26.0000  26.0000
...       ...         ...     ...         ...      ...      ...
2277  Topic20    0.000004     भाई   22.533638  -2.3736  -8.2453
3639  Topic20    0.000004  सुन्दर   17.650585  -2.1294  -8.2454
1352  Topic20    0.000004   देखकर   20.578547  -2.2830  -8.2455
1163  Topic20    0.000004    तेरे   16.670115  -2.0732  -8.2463
1348  Topic20    0.000004      दे   20.581808  -2.2840  -8.2463

[1475 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
222       1  1.019708       आया
405       1  0.987354        ओर
448       1  0.999858      करके
511       1  1.019359       काम
759       1  0.978390         च
940       1  1.018803     जल्दी
1136      1  1.019442      तुझे
1149      1  0.981663     तुल्य
1163      1  1.019789      तेरे
1348      1  1.020319        दे
1352      1  1.020480     देखकर
1355      1  1.019409     देखता
1364      1  1.020304      देखा
1848      1  1.020457       पास
2215      1  0.984952      बोला
2241      1  1.019821       भया
2277      1  1.020696       भाई
2374      1  0.978283        मन
2447      1  1.019658   महात्मा
2602      1  0.984743      मुझे
2649      1  1.020089      मेरी
2651      1  1.020729      मेरे
2655      1  1.019982     मैंने
2728      1  0.983317     युक्त
2730      1  1.019563     युद्ध
2742      1  1.019636     योग्य
2813      1  0.984777    राक्षस
2837      1  0.989752  राक्षसों
2854      1  1.020148      राजा
2874      1  0.997082       राम
2897      1  0.999898      रावण
2947      1  1.020277   लक्ष्मण
3032      1  1.001925       वचन
3078      1  0.990666     वाक्य
3086      1  0.999852      वानर
3099      1  1.018431    वानरों
3111      1  1.019647      वाला
3168      1  0.991598    विभीषण
3328      1  1.019370       शोक
3455      1  1.000222        सब
3487      1  0.992817    समुद्र
3597      1  1.019101       सिर
3603      1  1.001150      सीता
3622      1  1.019666   सुग्रीव
3628      1  0.994144     सुनकर
3639      1  1.019796    सुन्दर
3662      1  0.995708      सेना
3752      1  0.997686    हनुमान
3814      1  1.019727       हूं, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 16, 9, 3, 13, 15, 4, 10, 7, 5, 11, 12, 1, 20, 17, 18, 14, 6, 2, 8])